# References

* https://deepsense.io/deep-learning-right-whale-recognition-kaggle/
* http://matthewearl.github.io/2016/05/06/cnn-anpr/
* http://cs231n.stanford.edu/reports/2016/pdfs/285_Report.pdf
* https://www.kaggle.com/narae78/the-nature-conservancy-fisheries-monitoring/fish-detection/run/461339
